<a href="https://colab.research.google.com/github/harsh-hy/DeepLearning/blob/main/MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, _), (_,_ ) = tf.keras.datasets.mnist.load_data()

x_train = (x_train.astype("float32") - 127.5) / 127.5
x_train = x_train.reshape(x_train.shape[0] , 784)

In [ ]:
def build_generator():
  model= tf.keras.Sequential([
      layers.Dense(128, input_shape=(100,), activation="relu"),
      layers.Dense(784, activation="tanh")
      ])
  return model

In [ ]:
def build_discriminator():
  model= tf.keras.Sequential([
      layers.Dense(128, input_shape=(784,), activation="relu"),
      layers.Dense(1, activation="sigmoid")
      ])
  return model

In [ ]:
generator= build_generator()
discriminator= build_discriminator()

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
discriminator.trainable= False

In [ ]:
gan_input= tf.keras.Input(shape=(100,))
generated_img= generator(gan_input)
gan_output= discriminator(generated_img)
gan= tf.keras.Model(gan_input, gan_output)
gan.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
def train_gan(epochs=3000, batch_size=128):
  half_batch= batch_size // 2
  for epoch in range(epochs):
    idx= np.random.randint(0, x_train.shape[0], half_batch)
    real_imgs= x_train[idx]

    noise= np.random.normal(0, 1, (half_batch, 100))
    fake_imgs= generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

    noise= np.random.normal(0, 1, (batch_size, 100))
    valid_y= np.ones((batch_size, 1))
    g_loss= gan.train_on_batch(noise, np.ones((batch_size, 1)))

    if epoch % 200 == 0:
      print(f"Epoch: {epoch} | D Loss: {d_loss[0]:.4f}, acc: {100*d_loss[1]:.2f}% | G Loss: {g_loss:.4f}")
      show_generated_images(generator)

In [ ]:
def show_generated_images(generator, num_images=10):
  noise= np.random.normal(0, 1, (num_images, 100))
  generated_images= generator.predict(noise)
  generated_images= 0.5*generated_images + 0.5

  plt.figure(figsize=(4,4))
  for i in range(num_images):
    plt.subplot(4,4, i+1)
    plt.imshow(generated_images[i].reshape(28,28), cmap='gray')
    plt.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
train_gan(epochs=2000, batch_size=128)